In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import load_dataset
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

In [28]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("¿CUDA disponible?:", torch.cuda.is_available())
print("Número de GPUs:", torch.cuda.device_count())
print("Nombre de la GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No hay GPU disponible")

¿CUDA disponible?: True
Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [29]:
def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre   : {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()  # Verifica antes de entrenar

Memoria GPU asignada: 585.35 MB
Memoria GPU reservada: 630.00 MB
Memoria libre   : 3510.21 MB


In [30]:
path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\obras_socraticos_procesados.csv"
df_obras = pd.read_csv(path)


In [31]:
obras = df_obras["obras"]
print(obras)

0     clitofon, acusado por sócrates de haber censur...
1     la correspondencia que sigue comprende trece c...
2     ¿qué es lo justo? este es el comienzo , y el a...
3     hé aquí otra serie de preguntas y respuestas d...
4     la vida no es más que un viaje; y es natural, ...
5     el , muy superior a los diálogos inmediatament...
6     este diálogo no deja de tener analogía con el ...
7     quiere decir complemento o conclusión de las l...
8     timeo, nacido entre los locrios epicefirianos ...
9     platón, o el autor, cualquiera que él sea, de ...
10    un pormenor interesante relativo a hiparco, hi...
11    demodoco, padre del joven teages, suplica a só...
12    ateniense,si alguno usurpa, cerca de un gobier...
13    no debe orarse ligeramente. dirigir a los dios...
14    sin gracia en la forma, sin verdad y sin inter...
15    ateniense,se trata ahora de formar las leyes c...
16    ateniense,el orden de materias nos conduce a l...
17    ateniense,el orden natural de nuestras ley

In [32]:
from datasets import Dataset

# Convertir Dataframe a Dataset
dataset_obras = Dataset.from_pandas(df_obras[["obras"]])

In [33]:
print(dataset_obras)

Dataset({
    features: ['obras'],
    num_rows: 51
})


In [34]:
print(type(dataset_obras))
print(dataset_obras.column_names)
print(dataset_obras.features)
print(dataset_obras[1]) 

<class 'datasets.arrow_dataset.Dataset'>
['obras']
{'obras': Value(dtype='string', id=None)}
{'obras': 'la correspondencia que sigue comprende trece cartas, dirigidas una por dion a dionisio y las otras doce por platón a diferentes personajes contemporáneos.,las doce cartas de platón se dividen de esta manera: tres a dionisio, una a dion, dos a los parientes y amigos de dion, una a aristodoro, amigo de dion, dos a arquitas, una a pérdicas, una a hermias, erasmo y coriseo, y en fin, una a leodamas.,las primeras, que hacen relación a las cosas de sicilia y a los viajes de platón, son las que tienen algún interés histórico; en cuanto al interés filosófico y literario a todas les falta igualmente. bajo cualquier punto de vista que se las considere, estas cartas, aun sin exceptuar la sétima, no son en modo alguno dignas de platón.,\xa0,\xa0,\xa0,\xa0,\xa0,\xa0,'}


In [35]:
# Nombre del modelo y ruta del checkpoint
model_name = "PlanTL-GOB-ES/gpt2-base-bne"
#checkpoint_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\checkpoints\checkpoint-70"

In [36]:
# Cargar el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name, unk_token="[UNK]")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained(model_name) # cambiar luego a checkpoint path
model.resize_token_embeddings(len(tokenizer))

Embedding(50265, 768)

In [37]:
model.config.max_position_embeddings = 512

In [12]:
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(512, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50265, bias=False)
)

In [38]:
def dividir_en_chunks(texto, tokenizer, max_length=512):
    # Tokenizar el texto sin truncarlo
    tokens = tokenizer.encode(texto, truncation=False)
    
    # Dividir en chunks de tamaño max_length
    chunks = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    
    # Convertir los tokens de vuelta a texto
    # Convertimos de nuevo los tokens a texto para que el tokenizer los procese correctamente
    chunk_texts = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]
    
    return chunk_texts

In [39]:
def preparar_dataset(dataset_obras, tokenizer, max_length=512):
    # Para cada obra, dividirla en chunks
    chunks = []
    for obra in dataset_obras["obras"]:
        chunks.extend(dividir_en_chunks(obra, tokenizer, max_length))
    
    # Crear un nuevo dataset con los chunks
    return Dataset.from_dict({"obras": chunks})

# Preprocesar dataset: dividir las obras en chunks
dataset_obras_dividido = preparar_dataset(df_obras, tokenizer)

# Verifica que la estructura del dataset ahora esté basada en chunks
print(dataset_obras_dividido)

Dataset({
    features: ['obras'],
    num_rows: 57
})


In [40]:
def tokenization(dataset_obras):
    encodings = tokenizer(
        dataset_obras["obras"],
        truncation=True,
        padding="max_length",
        max_length=1024
    )

    encodings["labels"] = encodings["input_ids"].copy()

    # Convertimos a -100 los tokens de padding para que no contribuyan a la pérdida
    encodings["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in encodings["labels"]
    ]

    return {"input_ids": encodings["input_ids"], 
            "attention_mask": encodings["attention_mask"], 
            "labels": encodings["labels"]}

# Aplicar tokenización al dataset dividido en chunks
tokenized_obras = dataset_obras_dividido.map(tokenization, batched=True)

# Asegúrate de revisar el nuevo dataset tokenizado
print(tokenized_obras)

Map: 100%|██████████| 57/57 [00:00<00:00, 215.34 examples/s]

Dataset({
    features: ['obras', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 57
})


In [41]:
print(tokenized_obras)

Dataset({
    features: ['obras', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 57
})


In [42]:
print(type(tokenized_obras))
print(tokenized_obras.column_names)
print(tokenized_obras.features)
print(tokenized_obras[1]) 

<class 'datasets.arrow_dataset.Dataset'>
['obras', 'input_ids', 'attention_mask', 'labels']
{'obras': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'obras': 'la correspondencia que sigue comprende trece cartas, dirigidas una por dion a dionisio y las otras doce por platón a diferentes personajes contemporáneos.,las doce cartas de platón se dividen de esta manera: tres a dionisio, una a dion, dos a los parientes y amigos de dion, una a aristodoro, amigo de dion, dos a arquitas, una a pérdicas, una a hermias, erasmo y coriseo, y en fin, una a leodamas.,las primeras, que hacen relación a las cosas de sicilia y a los viajes de platón, son las que tienen algún interés histórico; en cuanto al interés filosófico y literario a todas les falta igualmente. bajo cual

In [47]:
# Convertir a formato PyTorch
tokenized_obras.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Configurar token de padding
model.config.pad_token_id = tokenizer.pad_token_id

In [46]:
print(type(tokenized_obras))
print(tokenized_obras.column_names)
print(tokenized_obras.features)
print(tokenized_obras[1]) 

<class 'datasets.arrow_dataset.Dataset'>
['obras', 'input_ids', 'attention_mask', 'labels']
{'obras': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
{'input_ids': tensor([    2,   407, 25980,  ..., 50264, 50264, 50264]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([    2,   407, 25980,  ...,  -100,  -100,  -100])}


In [44]:
import os

# Definir rutas
modelo_final_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\src\modelo_obras"
checkpoints_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\checkpoints"

# Asegurar que las carpetas existen
os.makedirs(modelo_final_path, exist_ok=True)
os.makedirs(checkpoints_path, exist_ok=True)

In [45]:
# Configuración de entrenamiento obras
training_args = TrainingArguments(
    output_dir=checkpoints_path,
    evaluation_strategy="no",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",
    remove_unused_columns=False,
    gradient_accumulation_steps=2, # Acumular gradientes cada 2 pasos
    learning_rate=1e-5,          # Tasa de aprendizaje ajustada
)

# Inicializar Trainer con dataset tokenizado
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_obras  # Usamos el dataset tokenizado
)

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
# Entrenar modelo obras
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [22]:
# Guardar el modelo
model.save_pretrained(modelo_final_path)

# Guardar el tokenizador
tokenizer.save_pretrained(modelo_final_path)

('C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\tokenizer_config.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\special_tokens_map.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\vocab.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\merges.txt',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\added_tokens.json',
 'C:\\Users\\Lucas\\Desktop\\Socrates\\llm_proyect\\src\\modelo_obras\\tokenizer.json')

In [23]:
input_text = "¿Qué es la justicia?"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

¿Qué es la justicia? 
